इस लेख में शामिल जप्टर नोटबुक [अध्याय 4 कोड आधार](https://github.com/datawhalechina/learn-nlp-with-transformers/tree/main/docs/%E7%AF%87%E7%AB) में है %A04 -%E4%BD%BF%E7%94%A8ट्रांसफॉर्मर%E8%A7%A3%E5%86%B3NLP%E4%BB%BB%E5%8A%A1)।

इस ट्यूटोरियल को सीधे खोलने के लिए Google Colab नोटबुक का उपयोग करने की अनुशंसा की जाती है, ताकि आप प्रासंगिक डेटा सेट और मॉडल जल्दी से डाउनलोड कर सकें।
यदि आप इस नोटबुक को Google के कोलाब में खोल रहे हैं, तो आपको ट्रांसफॉर्मर और 🤗डेटासेट लाइब्रेरी इंस्टॉल करने की आवश्यकता हो सकती है। इसे स्थापित करने के लिए निम्नलिखित कमांड को अनकम्मेंट करें।

In [ ]:
! pip install datasets transformers rouge-score nltk

वितरित प्रशिक्षण के लिए, कृपया [यहां] (https://github.com/huggingface/transformers/tree/master/examples/seq2seq) देखें।

सारांश उत्पादन कार्य को हल करने के लिए ट्रांसफार्मर मॉडल को ठीक से ट्यून करना

इस नोटबुक में, हम दिखाएंगे कि सारांश पीढ़ी कार्य को हल करने के लिए [🤗 ट्रांसफॉर्मर्स] (https://github.com/huggingface/transformers) में पूर्व-प्रशिक्षित मॉडल को कैसे ठीक किया जाए। हम [XSum डेटासेट](https://arxiv.org/pdf/1808.08745.pdf) डेटासेट का उपयोग करते हैं। इस डेटा सेट में बीबीसी लेख और संबंधित सारांश शामिल हैं। नीचे एक उदाहरण है:

![सारांशीकरण कार्य पर विजेट अनुमान](https://github.com/huggingface/notebooks/blob/master/examples/images/summarization.png?raw=1)

सारांश निर्माण कार्य के लिए, हम दिखाएंगे कि ट्रांसफॉर्मर में ट्रेनर इंटरफ़ेस का उपयोग करके डेटासेट को आसानी से कैसे लोड किया जाए और संबंधित प्रशिक्षण के लिए मॉडल को कैसे ठीक किया जाए।

In [ ]:
model_checkpoint = "t5-small"

जब तक पूर्व-प्रशिक्षित ट्रांसफार्मर मॉडल में seq2seq संरचना की मुख्य परत होती है, यह नोटबुक सैद्धांतिक रूप से किसी भी सारांश पीढ़ी कार्य को हल करने के लिए विभिन्न प्रकार के ट्रांसफार्मर मॉडल का उपयोग कर सकता है। यहां, हम [`t5-small`](https://huggingface.co/t5-small) मॉडल चेकपॉइंट का उपयोग करते हैं।

## डेटा डाउनलोड करें

हम डेटा और संबंधित मूल्यांकन विधियों को लोड करने के लिए [🤗 डेटासेट](https://github.com/huggingface/datasets) लाइब्रेरी का उपयोग करेंगे। डेटा लोडिंग और मूल्यांकन मोड लोडिंग के लिए केवल लोड_डेटासेट और लोड_मेट्रिक के सरल उपयोग की आवश्यकता होती है।

In [ ]:
from datasets import load_dataset, load_metric

raw_datasets = load_dataset("xsum")
metric = load_metric("rouge")

इसके अलावा, आप हमारे द्वारा प्रदान किए गए [लिंक] (https://gas.graviti.cn/dataset/datawhale/Xsum) से डेटा को डाउनलोड और डीकंप्रेस भी कर सकते हैं, और डीकंप्रेस्ड json फ़ाइल और `bbc-summary-data को संयोजित कर सकते हैं `फ़ोल्डर को `docs/Chapter 4-Using Transformers to Solve NLP Tasks/datasets/xsum` निर्देशिका में कॉपी करें, और फिर इसे निम्नलिखित कोड के साथ लोड करें। उसी तरह, आप हमारे द्वारा प्रदान की जाने वाली स्क्रिप्ट लोडिंग मूल्यांकन पद्धति `रूज` का भी उपयोग कर सकते हैं।

In [ ]:
import os

data_path = './dataset/xsum/'
path = os.path.join(data_path, 'xsum.py')
cache_dir = os.path.join(data_path, 'cache')
data_files = {'data': data_path}
dataset = load_dataset(path, data_files=data_files, cache_dir=cache_dir)

metric_path = './dataset/metrics/rouge.py'
metric = load_metric(metric_path)

यह डेटासेट ऑब्जेक्ट स्वयं एक [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict) डेटा संरचना है, प्रशिक्षण सेट, सत्यापन सेट और परीक्षण सेट के लिए, आपको केवल इसकी आवश्यकता है संबंधित डेटा का उपयोग कुंजी (ट्रेन, सत्यापन, परीक्षण) का उपयोग करके प्राप्त किया जा सकता है।

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 204045
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11332
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11334
    })
})

डेटा विभाजन कुंजी (ट्रेन, सत्यापन, या परीक्षण) और सबस्क्रिप्ट को देखते हुए, आप डेटा देख सकते हैं:

In [ ]:
raw_datasets["train"][0]

{'document': 'Recent reports have linked some France-based players with returns to Wales.\n"I\'ve always felt - and this is with my rugby hat on now; this is not region or WRU - I\'d rather spend that money on keeping players in Wales," said Davies.\nThe WRU provides £2m to the fund and £1.3m comes from the regions.\nFormer Wales and British and Irish Lions fly-half Davies became WRU chairman on Tuesday 21 October, succeeding deposed David Pickering following governing body elections.\nHe is now serving a notice period to leave his role as Newport Gwent Dragons chief executive after being voted on to the WRU board in September.\nDavies was among the leading figures among Dragons, Ospreys, Scarlets and Cardiff Blues officials who were embroiled in a protracted dispute with the WRU that ended in a £60m deal in August this year.\nIn the wake of that deal being done, Davies said the £3.3m should be spent on ensuring current Wales-based stars remain there.\nIn recent weeks, Racing Metro fla

यह समझने के लिए कि डेटा कैसा दिखता है, निम्न फ़ंक्शन प्रदर्शित करने के लिए डेटा सेट से यादृच्छिक रूप से कुछ उदाहरणों का चयन करेगा।

In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=2):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(raw_datasets["train"])

मेट्रिक [`datasets.Metric`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Metric) वर्ग का एक उदाहरण है। मीट्रिक और उपयोग के उदाहरण देखें:

In [ ]:
metric

Metric(name: "rouge", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}, usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each predictions
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLSum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/datasets/issues/617
    use_stemmer: Bool indicating whether Porter stemmer should be used to strip word suffixes.
    use_agregator: Return aggregates if this is set to True
Retu

हम स्कोर की गणना करने के लिए पूर्वानुमानों और लेबलों की तुलना करने के लिए `गणना` पद्धति का उपयोग करते हैं। पूर्वानुमान और लेबल दोनों को एक सूची बनाने की आवश्यकता है। विशिष्ट प्रारूप के लिए नीचे दिया गया उदाहरण देखें:

In [ ]:
fake_preds = ["hello there", "general kenobi"]
fake_labels = ["hello there", "general kenobi"]
metric.compute(predictions=fake_preds, references=fake_labels)

{'rouge1': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rouge2': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rougeL': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rougeLsum': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0))}

## डेटा प्रीप्रोसेसिंग

मॉडल में डेटा फीड करने से पहले, हमें डेटा को प्रीप्रोसेस करना होगा। प्रीप्रोसेसिंग टूल को टोकननाइज़र कहा जाता है। टोकनाइज़र पहले इनपुट को टोकनाइज़ करता है, फिर टोकन को प्री-मॉडल में आवश्यक संबंधित टोकन आईडी में परिवर्तित करता है, और फिर इसे मॉडल द्वारा आवश्यक इनपुट प्रारूप में परिवर्तित करता है।

डेटा प्रीप्रोसेसिंग के उद्देश्य को प्राप्त करने के लिए, हम अपने टोकननाइज़र को इंस्टेंट करने के लिए AutoTokenizer.from_pretrained विधि का उपयोग करते हैं, जो सुनिश्चित करता है:

- हमें एक टोकननाइज़र मिलता है जो पूर्व-प्रशिक्षित मॉडल वन-टू-वन से मेल खाता है।
- निर्दिष्ट मॉडल चेकपॉइंट के अनुरूप टोकननाइज़र का उपयोग करते समय, हम मॉडल द्वारा आवश्यक शब्दावली लाइब्रेरी भी डाउनलोड करते हैं, सटीक होने के लिए, टोकन शब्दावली।


डाउनलोड किए गए टोकन शब्दावली को कैश किया जाएगा ताकि दोबारा उपयोग करने पर इसे दोबारा डाउनलोड न किया जा सके।

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

डिफ़ॉल्ट रूप से, उपरोक्त कॉल 🤗 टोकनाइज़र लाइब्रेरी से तेज़ टोकनाइज़र (रस्ट द्वारा समर्थित) में से एक का उपयोग करेगी।

टोकननाइज़र या तो एक टेक्स्ट या टेक्स्ट की एक जोड़ी को प्रीप्रोसेस कर सकता है। टोकननाइज़र प्रीप्रोसेसिंग के बाद प्राप्त डेटा प्री-ट्रेनिंग मॉडल इनपुट प्रारूप से मिलता है।

In [ ]:
tokenizer("Hello, this one sentence!")

{'input_ids': [8774, 6, 48, 80, 7142, 55, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

ऊपर देखी गई टोकन आईडी, जो कि इनपुट_आईडी हैं, आम तौर पर पूर्व-प्रशिक्षित मॉडल के नाम के साथ बदलती रहती हैं। इसका कारण यह है कि विभिन्न पूर्व-प्रशिक्षण मॉडल पूर्व-प्रशिक्षण के दौरान अलग-अलग नियम निर्धारित करते हैं। लेकिन जब तक टोकननाइजर और मॉडल के नाम सुसंगत हैं, तब तक टोकननाइजर द्वारा पूर्व-संसाधित इनपुट प्रारूप मॉडल की जरूरतों को पूरा करेगा। प्रीप्रोसेसिंग के बारे में अधिक जानकारी के लिए, कृपया [यह ट्यूटोरियल](https://huggingface.co/transformers/preprocessing.html) देखें

किसी वाक्य को टोकनाइज़ करने के अलावा, हम वाक्यों की सूची को भी टोकनाइज़ कर सकते हैं।

In [ ]:
tokenizer(["Hello, this one sentence!", "This is another sentence."])

{'input_ids': [[8774, 6, 48, 80, 7142, 55, 1], [100, 19, 430, 7142, 5, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

नोट: मॉडल के लिए अनुवादित लक्ष्य तैयार करने के लिए, हम लक्ष्य के अनुरूप विशेष टोकन को नियंत्रित करने के लिए `as_target_tokenizer` का उपयोग करते हैं:

In [ ]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["Hello, this one sentence!", "This is another sentence."]))

{'input_ids': [[8774, 6, 48, 80, 7142, 55, 1], [100, 19, 430, 7142, 5, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}


यदि आप T5 पूर्व-प्रशिक्षित मॉडल की चौकियों का उपयोग कर रहे हैं, तो आपको विशेष उपसर्ग की जाँच करने की आवश्यकता है। T5 मॉडल को किए जाने वाले विशिष्ट कार्यों को बताने के लिए विशेष उपसर्गों का उपयोग करता है। विशिष्ट उपसर्गों के उदाहरण इस प्रकार हैं:

In [ ]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

अब हम अपना प्रीप्रोसेसर फ़ंक्शन बनाने के लिए सब कुछ एक साथ रख सकते हैं। जब हम नमूने को प्रीप्रोसेस करते हैं, तो हम यह सुनिश्चित करने के लिए `truncation=True` पैरामीटर का भी उपयोग करेंगे कि हमारे अत्यधिक लंबे वाक्यों को छोटा कर दिया गया है। डिफ़ॉल्ट रूप से, हम छोटे वाक्यों को स्वचालित रूप से पैड कर देंगे।

In [ ]:
max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

# लक्ष्य के लिए टोकननाइज़र सेटअप करें
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

उपरोक्त प्रीप्रोसेसिंग फ़ंक्शन एक नमूना या एकाधिक नमूना उदाहरणों को संसाधित कर सकता है। यदि कई नमूनों को संसाधित किया जाता है, तो कई नमूनों को पूर्व-संसाधित करने के बाद परिणामों की एक सूची वापस कर दी जाती है।

In [ ]:
preprocess_function(raw_datasets['train'][:2])

{'input_ids': [[21603, 10, 17716, 2279, 43, 5229, 128, 1410, 18, 390, 1508, 28, 5146, 12, 10256, 5, 96, 196, 31, 162, 373, 1800, 3, 18, 11, 48, 19, 28, 82, 22209, 3, 547, 30, 230, 117, 48, 19, 59, 1719, 42, 549, 8503, 3, 18, 27, 31, 26, 1066, 1492, 24, 540, 30, 2627, 1508, 16, 10256, 976, 243, 28571, 5, 37, 549, 8503, 795, 17586, 51, 12, 8, 3069, 11, 3996, 13606, 51, 639, 45, 8, 6266, 5, 18263, 10256, 11, 2390, 11, 7262, 10371, 7, 3971, 18, 17114, 28571, 1632, 549, 8503, 13404, 30, 2818, 1401, 1797, 6, 7229, 53, 20, 12151, 1955, 8356, 49, 53, 826, 3, 19585, 643, 9768, 5, 216, 19, 230, 3122, 3, 9, 2103, 1059, 12, 1175, 112, 1075, 38, 24260, 350, 16103, 10282, 7, 5752, 4297, 227, 271, 3, 11060, 30, 12, 8, 549, 8503, 1476, 16, 1600, 5, 28571, 47, 859, 8, 1374, 5638, 859, 10282, 7, 6, 411, 7, 2026, 63, 7, 6, 14586, 7677, 11, 26911, 2419, 7, 4298, 113, 130, 10960, 52, 26786, 16, 3, 9, 813, 11674, 11044, 28, 8, 549, 8503, 24, 3492, 16, 3, 9, 3996, 3328, 51, 1154, 16, 1660, 48, 215, 5, 86, 8,

इसके बाद, डेटासेट में सभी नमूने पूर्व-संसाधित होते हैं। प्रसंस्करण की विधि मैप फ़ंक्शन का उपयोग करना और सभी नमूनों पर प्रीप्रोसेसिंग फ़ंक्शन तैयार_ट्रेन_फीचर्स को लागू करना है।

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

इससे भी बेहतर, अगली बार संसाधित होने पर पुनर्गणना से बचने के लिए लौटाए गए परिणाम स्वचालित रूप से कैश किए जाएंगे (लेकिन कृपया यह भी ध्यान दें कि यदि इनपुट बदल दिया जाता है, तो यह कैश से प्रभावित हो सकता है!)। डेटासेट लाइब्रेरी फ़ंक्शन यह निर्धारित करने के लिए इनपुट पैरामीटर का पता लगाएगा कि क्या परिवर्तन हैं। यदि कोई परिवर्तन नहीं हैं, तो कैश्ड डेटा का उपयोग किया जाएगा। यदि परिवर्तन हैं, तो डेटा को पुन: संसाधित किया जाएगा। लेकिन यदि इनपुट पैरामीटर अपरिवर्तित रहते हैं और आप इनपुट बदलना चाहते हैं, तो कैश को साफ़ करना और समायोजित करना सबसे अच्छा है। साफ़ करने का तरीका `load_from_cache_file=False` पैरामीटर का उपयोग करना है। इसके अलावा, ऊपर इस्तेमाल किया गया `बैच = ट्रू` पैरामीटर टोकननाइज़र की एक विशेषता है, जिसके बारे में माना जाता है कि यह एक ही समय में समानांतर में इनपुट को संसाधित करने के लिए कई थ्रेड्स का उपयोग करता है।

## मॉडल को फाइन-ट्यून करें

अब जब डेटा तैयार हो गया है, तो हमें अपने पूर्व-प्रशिक्षित मॉडल को डाउनलोड और लोड करना होगा, और फिर पूर्व-प्रशिक्षित मॉडल को फाइन-ट्यून करना होगा। चूँकि हम seq2seq कार्य कर रहे हैं, हमें एक मॉडल वर्ग की आवश्यकता है जो इस कार्य को हल कर सके। हम वर्ग `AutoModelForSeq2SeqLM` का उपयोग करते हैं। टोकननाइज़र के समान, `from_pretrained` विधि भी हमें मॉडल को डाउनलोड और लोड करने में मदद कर सकती है, यह मॉडल को कैश भी करेगी ताकि मॉडल बार-बार डाउनलोड न हो।

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

चूँकि हमारा फाइन-ट्यूनिंग कार्य एक seq2seq कार्य है, और हम एक पूर्व-प्रशिक्षित seq2seq मॉडल लोड कर रहे हैं, मॉडल लोड करते समय हमें कुछ बेमेल तंत्रिका नेटवर्क मापदंडों को फेंकने के लिए प्रेरित नहीं किया जाएगा (उदाहरण के लिए: पूर्व का तंत्रिका नेटवर्क प्रमुख) -प्रशिक्षित भाषा मॉडल को फेंक दिया गया, जबकि मशीनी अनुवाद के लिए तंत्रिका नेटवर्क प्रमुख को बेतरतीब ढंग से आरंभ किया गया)।

`Seq2SeqTrainer` प्रशिक्षण उपकरण प्राप्त करने के लिए, हमें 3 तत्वों की भी आवश्यकता है, जिनमें से सबसे महत्वपूर्ण है प्रशिक्षण सेटिंग्स/पैरामीटर [`Seq2SeqTrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html# ट्रांसफार्मर.Seq2SeqTrainingArguments)। इस प्रशिक्षण सेटिंग में वे सभी विशेषताएँ शामिल हैं जो प्रशिक्षण प्रक्रिया को परिभाषित करती हैं

In [ ]:
batch_size = 16
args = Seq2SeqTrainingArguments(
    "test-summarization",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
)

उपरोक्त मूल्यांकन_रणनीति = "युग" पैरामीटर प्रशिक्षण कोड बताता है: हम प्रत्येक युग के लिए एक सत्यापन मूल्यांकन करेंगे।

उपरोक्त बैच_आकार इस नोटबुक से पहले परिभाषित किया गया था।

चूँकि हमारा डेटा सेट अपेक्षाकृत बड़ा है और `Seq2SeqTrainer` मॉडलों को सहेजना जारी रखेगा, हमें इसे अधिकतम `save_total_limit=3` मॉडलों को सहेजने के लिए कहना होगा।

अंत में, हमें अपने संसाधित इनपुट को मॉडल में फीड करने के लिए एक डेटा कोलेटर की आवश्यकता होती है।

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

`Seq2SeqTrainer` की स्थापना के बाद एक आखिरी काम बचा है, वह यह है कि हमें मूल्यांकन पद्धति को परिभाषित करने की आवश्यकता है। हम मूल्यांकन पूरा करने के लिए `मीट्रिक` का उपयोग करते हैं। मूल्यांकन में मॉडल पूर्वानुमानों को फीड करने से पहले, हम कुछ डेटा पोस्ट-प्रोसेसिंग भी करेंगे:

In [ ]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
# लेबल में -100 बदलें क्योंकि हम उन्हें डिकोड नहीं कर सकते।
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
# रूज प्रत्येक वाक्य के बाद एक नई पंक्ति की अपेक्षा करता है
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
# कुछ परिणाम निकालें
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
# औसत उत्पन्न लंबाई जोड़ें
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

अंत में, सभी पैरामीटर/डेटा/मॉडल को `Seq2SeqTrainer` पर पास करें

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

प्रशिक्षण को बेहतर बनाने के लिए `ट्रेन` विधि को कॉल करें।

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len,Runtime,Samples Per Second
1,2.721100,2.479327,28.300900,7.721100,22.243000,22.249600,18.822500,326.333800,34.725000


TrainOutput(global_step=12753, training_loss=2.7692033505520146, metrics={'train_runtime': 4909.3835, 'train_samples_per_second': 2.598, 'total_flos': 7.774481450954342e+16, 'epoch': 1.0, 'init_mem_cpu_alloc_delta': 335248, 'init_mem_gpu_alloc_delta': 242026496, 'init_mem_cpu_peaked_delta': 18306, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 2637782, 'train_mem_gpu_alloc_delta': 728138240, 'train_mem_cpu_peaked_delta': 138226182, 'train_mem_gpu_peaked_delta': 14677017088})

अंत में, यह देखना न भूलें कि मॉडल कैसे अपलोड करें, मॉडल को](https://huggingface.co/transformers/model_sharing.html) से [🤗 मॉडल हब](https://huggingface.co/models) पर अपलोड करें। . फिर आप मॉडल नाम का उपयोग करके सीधे अपने मॉडल का उपयोग कर सकते हैं जैसे आपने इस नोटबुक की शुरुआत में किया था।